In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn

%matplotlib inline

On-Base Percentage (OBP) = Hits + Walks(BB) + Hit By Pitch(HBP) / At Bats(AB) + Walks + HBP + Sacrifice Flies

Note = Sacrifice Flies stats don't start until 1954.

Before that, SH = SF + Bunts

In [39]:
column_id = [0, 13, 14]
bio = pd.read_csv('data/Master.csv', usecols=column_id)
len(bio)

18846

In [14]:
bio = bio.rename(columns={'nameFirst': 'FirstName', 'nameLast': 'LastName'})

In [15]:
bio.head()

,playerID,FirstName,LastName
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad


In [41]:
column_id = [0, 3, 4]
salary = pd.read_csv('data/Salaries.csv', usecols=column_id)
len(salary)

25575

In [43]:
column_id = [0, 1, 5, 6]
fielding = pd.read_csv('data/Fielding.csv', usecols=column_id)
len(fielding)

170526

In [44]:
column_id = [0, 1, 5, 6, 8, 15, 18, 20]
batting = pd.read_csv('data/Batting.csv', usecols=column_id)
len(batting)

101332

In [45]:
data = pd.merge(batting, bio)
data = pd.merge(data, fielding)
data = pd.merge(data, salary)

In [46]:
len(data)

13709

In [65]:
data.head()

,playerID,yearID,G,AB,H,BB,HBP,SF,nameFirst,nameLast,POS,salary
0,johnto01,1985,12,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,600000
1,johnto01,1985,11,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,600000
2,johnto01,1986,13,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,440000
3,johnto01,1986,13,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,60000
4,johnto01,1987,33,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,350000


In [47]:
data_dropna = data.copy()

In [56]:
data_dropna = data_dropna.dropna()

In [57]:
len(data_dropna)

11562

In [60]:
data_dropna = data_dropna.drop_duplicates()

In [61]:
len(data_dropna)

11492

In [76]:
data_dropna['OBP'] = (data_dropna['H']+data_dropna['BB']+data_dropna['HBP'])/(data_dropna['AB']+data_dropna['BB']+data_dropna['HBP']+data_dropna['SF'])

In [77]:
data_dropna.head()

,playerID,yearID,G,AB,H,BB,HBP,SF,nameFirst,nameLast,POS,salary,OBP,OBP/Sal_Ratio,Ratio
8,carltst01,1985,16,28.0,5.0,1.0,0.0,1.0,Steve,Carlton,P,1075000,0.200000,1.860465e-07,1.860465e-07
9,carltst01,1986,16,34.0,7.0,1.0,0.0,0.0,Steve,Carlton,P,60000,0.228571,3.809524e-06,3.809524e-06
10,carltst01,1986,16,34.0,7.0,1.0,0.0,0.0,Steve,Carlton,P,1000000,0.228571,2.285714e-07,2.285714e-07
11,carltst01,1986,6,11.0,2.0,0.0,0.0,0.0,Steve,Carlton,P,60000,0.181818,3.030303e-06,3.030303e-06
12,carltst01,1986,6,11.0,2.0,0.0,0.0,0.0,Steve,Carlton,P,1000000,0.181818,1.818182e-07,1.818182e-07


In [78]:
data_dropna['Ratio'] = data_dropna['OBP']/data_dropna['salary']

In [79]:
moneyball = data_dropna[data_dropna['AB'] > 15]

In [81]:
moneyball.sort_values('Ratio', ascending=False).head()

,playerID,yearID,G,AB,H,BB,HBP,SF,nameFirst,nameLast,POS,salary,OBP,OBP/Sal_Ratio,Ratio
1664,gibbojo02,1986,8,19.0,9.0,3.0,0.0,0.0,John,Gibbons,C,65000,0.545455,0.000008,0.000008
1455,lakest01,1986,10,19.0,8.0,1.0,0.0,0.0,Steve,Lake,C,60000,0.450000,0.000008,0.000008
3335,jonesro02,1989,12,31.0,9.0,9.0,0.0,0.0,Ron,Jones,OF,74000,0.450000,0.000006,0.000006
3001,ripkebi01,1987,58,234.0,72.0,21.0,0.0,1.0,Billy,Ripken,2B,62500,0.363281,0.000006,0.000006
3021,thurmga01,1987,27,81.0,24.0,8.0,0.0,0.0,Gary,Thurman,OF,62500,0.359551,0.000006,0.000006


In [82]:
del moneyball['OBP/Sal_Ratio']

In [83]:
moneyball.head()

,playerID,yearID,G,AB,H,BB,HBP,SF,nameFirst,nameLast,POS,salary,OBP,Ratio
8,carltst01,1985,16,28.0,5.0,1.0,0.0,1.0,Steve,Carlton,P,1075000,0.200000,1.860465e-07
9,carltst01,1986,16,34.0,7.0,1.0,0.0,0.0,Steve,Carlton,P,60000,0.228571,3.809524e-06
10,carltst01,1986,16,34.0,7.0,1.0,0.0,0.0,Steve,Carlton,P,1000000,0.228571,2.285714e-07
16,ryanno01,1985,35,63.0,7.0,4.0,0.0,1.0,Nolan,Ryan,P,1350000,0.161765,1.198257e-07
17,ryanno01,1986,30,59.0,6.0,1.0,0.0,0.0,Nolan,Ryan,P,1125000,0.116667,1.037037e-07
